In [ ]:
!pip install -q torch
!pip install -q accelerate
!pip install -q peft
!pip install -q transformers
!pip install -q trl
!pip install -q -U bitsandbytes==0.46.0

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

When the pretraining is complete, auto-regressive models like Llama 2 can predict the next token in a sequence. However, this does not make them particularly useful assistants since they don’t reply to instructions. This is why we employ instruction tuning to align their answers with what humans expect. There are two main fine-tuning techniques:



Supervised Fine-Tuning (SFT): Models are trained on a dataset of instructions and responses. It adjusts the weights in the LLM to minimize the difference between the generated answers and ground-truth responses, acting as labels.

Reinforcement Learning from Human Feedback (RLHF): Models learn by interacting with their environment and receiving feedback. They are trained to maximize a reward signal (using PPO), which is often derived from human evaluations of model outputs.



In general, RLHF is shown to capture more complex and nuanced human preferences, but is also more challenging to implement effectively. Indeed, it requires careful design of the reward system and can be sensitive to the quality and consistency of human feedback. A possible alternative in the future is the Direct Preference Optimization (DPO) algorithm, which directly runs preference learning on the SFT model.

 we will perform SFT, but this raises a question: why does fine-tuning work in the first place?

fine-tuning will be of little help if the model has never seen the kind of data you’re interested in. However, if that’s the case, SFT can be extremely performant.

# llama 2 prompt template

`<s>` [INST] `<<SYS>>`
System prompt
`<</SYS>>`

User prompt [/INST] Model `</s>`

# loading dataset

In [ ]:
!pip install -q datasets
!huggingface-cli login

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}


# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

# setting the params

### LoRA recap in one line

LoRA (Low-Rank Adaptation) freezes a large, pre-trained weight matrix **W₀** and learns a low-rank update

$$
\Delta W \;=\; \frac{\alpha}{r}\;A\,B
$$

that is **added** to the original weights during the forward pass.

---

## What **`lora_alpha`** does

| Symbol                 | Default role                                                                           |
| ---------------------- | -------------------------------------------------------------------------------------- |
| **`r`**                | rank of the two small matrices $A\in\mathbb R^{d\times r},\;B\in\mathbb R^{r\times k}$ |
| **`α` (`lora_alpha`)** | *scaling factor* that controls the **magnitude** of the low-rank update                |

In code the forward pass of a LoRA-ised linear layer looks roughly like:

```python
def forward(x):
    original = x @ W0.T                     # frozen pre-trained weights
    lora_out = (x @ A) @ B                  # low-rank branch
    scaled   = (alpha / r) * lora_out       # <--  α/r  scaling
    return original + scaled                # add the adaptation
```

So **yes**:

1. The low-rank branch is computed.
2. It is multiplied by **`alpha / r`**.
3. The scaled result is **added to the main projection output** (for
   Q, K, V or FFN weights – wherever LoRA is inserted).

---

### Why use the scale?

* To keep the update’s variance comparable to the frozen weights.
* Original LoRA paper set **α = r** (so α/r = 1) by default; frameworks expose `lora_alpha` so you can dampen or amplify the adaptation:

  * **α < r** → gentler update, safer from overfitting.
  * **α > r** → stronger adaptation, may need smaller learning-rate.

---

lora_dropout is a regularisation knob inserted only on the LoRA branch to stop that low-rank update from over-fitting:

```python
drop    = nn.Dropout(p=lora_dropout)        # keeps p = 1-p_dropout tokens
lora_in = drop(x)                           # apply *after* layer-norm, before A
lora_out = (lora_in @ A) @ B
scaled   = (alpha / r) * lora_out
y        = original + scaled
```

What it does – randomly zero-outs rows of the mini-batch (or key/value heads, depending on the implementation) before they pass through the LoRA matrices
𝐴
,
𝐵
A,B.

Why – the backbone weights
𝑊
0
W
0
​
  stay intact; dropping activations only affects the learnable
Δ
𝑊
ΔW.
This nudges training to learn a low-rank update that is not overly reliant on any one token/head and keeps the adaptation small.

Typical default – lora_dropout = 0.0 (i.e., disabled) because LoRA already introduces relatively few new parameters; you enable it (e.g., 0.05 – 0.1) when you see the adaptation over-fitting on a small fine-tune set.

| Parameter      | Role in forward pass                                                             |
| -------------- | -------------------------------------------------------------------------------- |
| `lora_alpha`   | Scales the **magnitude** of the low-rank update $\alpha/r$                       |
| `lora_dropout` | Applies dropout **on the LoRA branch’s input** before $A$, adding regularisation |


### Where in the Transformer?

`A,B` blocks are typically attached to the **linear projections** inside self-attention (`W_Q`, `W_K`, `W_V`, `W_O`) and/or the feed-forward layers.
The scaling happens *inside those projections* – not at the entire attention output.

---

> **Bottom line:** `lora_alpha` is just a scalar that rescales the learned low-rank update before it is *added* to the frozen weight’s output. Bigger `alpha` ⇒ bigger influence of LoRA on the final activations.


# 🚀 LoRA & QLoRA — Everything You Need to Know

*A snack-size guide for interviewers, new teammates, or your future self.*

---

## 1 LoRA (Low-Rank Adaptation)

| What it solves | How it solves it |
|----------------|------------------|
| Fine-tuning huge ( ≫ 1 B ) models is **slow & memory-hungry**. | Freeze the giant weights **W₀** and learn a **tiny low-rank patch ΔW**. |

### 1.1 Core math

\[
\boxed{ \displaystyle
W \;=\; W_0 \;+\; \tfrac{\alpha}{r}\;A B
}\]

* `r` = small rank (8–64)  
* `α` =`lora_alpha` (gain)  
* `A∈ℝ^{d×r}` is **down-projector** (random init)  
* `B∈ℝ^{r×k}` is **up-projector** (-zero init → ΔW = 0 at step 0)

> **Why zeros for B?** Day-0 model behaves **exactly** like the base model.

---

### 1.2 Hyper-parameters in one table

| Name | Default hint | Role |
|------|--------------|------|
| `r` | 8 – 32 | Rank of the update (controls # LoRA params). |
| `lora_alpha` | often `2 × r` | Scales update: effective gain = `α/r`. |
| `lora_dropout` | `0 – 0.1` | Drop tokens *only on LoRA branch* → regularises ΔW. |

**Forward pass (pseudo-code)**  

```python
orig  = x @ W0.T                    # frozen path
l_in  = dropout(x, p=lora_dropout)  # maybe zero rows
lora  = (alpha / r) * (l_in @ A) @ B
y     = orig + lora

Only `A` & `B` get gradients ⇒ ≈ 0.2 % extra VRAM.

---

## 2 QLoRA (Quantized LoRA)

| Key trick                                                                                   | Result                                                        |
| ------------------------------------------------------------------------------------------- | ------------------------------------------------------------- |
| Store every frozen matrix in **4-bit** ints; de-quantise to 16-bit *just-in-time* for GEMM. | 8× VRAM save, yet math still uses fast tensor-core BF16/FP16. |

### 2.1 Where each tensor lives

| Tensor              | **Stored** | **During math**      | Trainable?   |
| ------------------- | ---------- | -------------------- | ------------ |
| Frozen weights `W₀` | **4-bit**  | de-quant → BF16/FP16 | ❌            |
| LoRA `A,B`          | BF16/FP16  | BF16/FP16            | ✅            |
| Final `lm_head`     | BF16/FP16  | BF16/FP16            | ✅ (optional) |
| Activations (`X`)   | BF16/FP16  | BF16/FP16            | n/a          |

> **Important**: No 4-bit number is multiplied directly—every int4 tile is inflated to 16-bit *inside* the fused kernel, multiplied, then discarded.

### 2.2 One GEMM in 6 micro-steps

1. Load 4-bit tile of `W₀` from GPU DRAM.
2. De-quantise → BF16 in shared memory.
3. Read LoRA `A,B` (already BF16).
4. Optional `lora_dropout` on `A`.
5. Compute ΔW = (α/r) · A B.
6. Tensor-core GEMM: `Y = X · (W₀ + ΔW)` → output BF16.

Memory never balloons because the BF16 copy of `W₀` is thrown away after the multiply.

---

## 3 `lm_head` — the Last Linear Layer

* **Where?** Right after the final Transformer block.
* **What?** Simple matrix multiply that turns hidden vector `h` into logits over the vocabulary.
* **Why keep it 16-bit?** It’s small, and quantising would inject noise straight into the soft-max.

---

## 4 FAQ (Lightning Round)

| Q                                   | A                                                                                  |
| ----------------------------------- | ---------------------------------------------------------------------------------- |
| *Do tokens live in 4-bit?*          | No. Activations are produced directly in BF16/FP16; only **weights** are 4-bit.    |
| *What does `lora_alpha` really do?* | It multiplies the LoRA update **after** the mat-mul. Bigger α ⇒ louder LoRA voice. |
| *Where is `lora_dropout` applied?*  | On the **input activations entering `A`**—never on the frozen path.                |
| *How are `A,B` initialised?*        | `A` small random; **`B = 0`** so step-0 ≡ base model.                              |
| *Is QLoRA slower?*                  | ≤ 3 % overhead—de-quant is fused with GEMM.                                        |
| *Can I tie `lm_head` & embeddings?* | Yes; weight-tying still works—just store the shared matrix in BF16.                |

---

### ✨ TL;DR

**LoRA**: add a tiny low-rank patch; freeze the rest.
**QLoRA**: store that frozen bulk in 4-bit, inflate to 16-bit for math, still train only the patch.
Memory drops \~8×; quality stays almost the same. Perfect for laptop-level fine-tuning!


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)